Escape the maze

In [3]:
package path;

import java.util.*;

public class Maze {

    public static class Pos {
        final int y;
        final int x;

        Pos(int y, int x) {
            this.y = y;
            this.x = x;
        }

        Pos add(Pos p) {
            return new Pos(y + p.y, x + p.x);
        }

        Pos delta(Pos p) {
            return new Pos(y - p.y, x - p.x);
        }

        @Override
        public int hashCode() {
            return x * 31 + y;
        }

        @Override
        public boolean equals(Object obj) {
            if ( obj instanceof Pos) {
                Pos p = (Pos) obj;
                return x == p.x && y == p.y;
            }
            return false;
        }

        @Override
        public String toString() {
            return "{ y=" + y + ", x = " + x +" }";
        }
    }

    private final char [][] arr;
    private final int xs;
    private final int ys;
    private final Pos start;

    final private Set<Character> dirSet = new HashSet<>(Arrays.asList('^', '<', 'v','>'));
    final private List<Pos> delta =
            Arrays.asList(new Pos(0,1), new Pos(0,-1), new Pos(1,0), new Pos(-1,0));

    public Maze(char [][] arr) {
        this.arr = arr;
        ys = arr.length;
        xs = arr[0].length;
        start = findStartPos();
    }

    private Pos findStartPos() {
        for ( int y = 0; y < ys; ++y ) {
            for ( int x = 0; x < xs; ++x ) {
                if ( dirSet.contains(arr[y][x]) ) {
                    return new Pos(y,x);
                }
            }
        }
        return new Pos(0,0);
    }

    private boolean isFreeStep(Pos p) {
        return ( arr[p.y][p.x] == ' ');
    }

    private boolean isExit(Pos p) {
        return p.y == 0 || p.x == 0 || p.y == ys-1 || p.x == xs-1;
    }

    private Pos getStepDirection(Pos p1, Pos p2, Pos prevDelta, LinkedList<Character> steps) {
        Pos d = p2.delta(p1);
        if ( prevDelta == null ) return d;
        if ( d.x != 0 ) {
            if (prevDelta.y == d.x) {
                steps.addFirst('R'); //right turn vert
                return d;
            } else if (d.x + prevDelta.y == 0 ) {
                steps.addFirst('L'); //left turn vert
                return d;
            }
        } else if ( d.y != 0 ) {
            if ( prevDelta.x == d.y ) {
                steps.addFirst('L'); //left turn hor
                return d;
            } else if ( prevDelta.x + d.y == 0 ) {
                steps.addFirst('R'); //right turn hor
                return d;
            }
        }
        return prevDelta; //same direction
    }

    private Pos getInitDir() {
        char startChar = arr[start.y][start.x];
        if ( startChar == '^' ) return new Pos(-1, 0);
        if ( startChar == 'v' ) return new Pos(1, 0);
        if ( startChar == '>' ) return new Pos(0, 1);
        return new Pos(0, -1);
    }

    private void getFirstTurn(Pos dir, LinkedList<Character> steps) {
        Pos startDir = getInitDir();
        if ( dir.x != 0 ) {
            if ( dir.x + startDir.x == 0 ) {
                steps.addFirst('B');
            } else if ( startDir.y == dir.x) {
                steps.addFirst('L');
            } else if ( startDir.y + dir.x == 0 ) {
                steps.addFirst('R');
            }
        } else {
            if ( dir.y + startDir.y == 0 ) {
                steps.addFirst('B');
            } else if ( startDir.x == dir.y) {
                steps.addFirst('R');
            } else if ( startDir.x + dir.y == 0 ) {
                steps.addFirst('L');
            }
        }
    }

    public List<Character> find() {
        if ( isExit(start) ) return Collections.emptyList();
        LinkedList<Pos> q = new LinkedList<>();
        q.add(start);
        HashMap<Pos, Pos> visited = new HashMap<>();
        visited.put(start, start);
        Pos exit = null;
        while ( q.size() > 0 && exit == null ) {
            Pos c = q.pollFirst();
            if ( isExit(c) ) {
                exit = c;
            } else {
                for (Pos d : delta) {
                    Pos np = c.add(d);
                    if (isFreeStep(np) && !visited.containsKey(np)) {
                        q.add(np);
                        visited.put(np, c);
                    }
                }
            }
        }
        if ( exit == null ) return Collections.emptyList();
        LinkedList<Character> steps = new LinkedList<>();
        Pos cur = exit;
        Pos prevDirection = null;
        while ( ! cur.equals(start) ) {
            Pos prev = visited.get(cur);
            prevDirection = getStepDirection(prev, cur, prevDirection, steps);
            steps.addFirst('F');
            cur = prev;
        }
        getFirstTurn(prevDirection, steps);
        return steps;

    }
    
    public static List<Character> escape(char[][] maze) {
        Maze m = new Maze(maze);
        return m.find();
    }

    public static List<char[][]> buildTests() {
        List<char[][]> basicMazes = new LinkedList<>();

        basicMazes.add(new char[][]{
                "# #".toCharArray(),
                " > ".toCharArray(),
                "# #".toCharArray()
        });
        basicMazes.add(new char[][]{
                "###########".toCharArray(),
                "#>        #".toCharArray(),
                "######### #".toCharArray()
        });
        basicMazes.add(new char[][]{
                "# #########".toCharArray(),
                "#        >#".toCharArray(),
                "###########".toCharArray()
        });
        basicMazes.add(new char[][]{
                "####### #".toCharArray(),
                "#>#   # #".toCharArray(),
                "#   #   #".toCharArray(),
                "#########".toCharArray()
        });
        basicMazes.add(new char[][]{
                "##########".toCharArray(),
                "#        #".toCharArray(),
                "#  ##### #".toCharArray(),
                "#  #   # #".toCharArray(),
                "#  #^# # #".toCharArray(),
                "#  ### # #".toCharArray(),
                "#      # #".toCharArray(),
                "######## #".toCharArray()
        });
        basicMazes.add(new char[][]{
                "#########################################".toCharArray(),
                "#<    #       #     #         # #   #   #".toCharArray(),
                "##### # ##### # ### # # ##### # # # ### #".toCharArray(),
                "# #   #   #   #   #   # #     #   #   # #".toCharArray(),
                "# # # ### # ########### # ####### # # # #".toCharArray(),
                "#   #   # # #       #   # #   #   # #   #".toCharArray(),
                "####### # # # ##### # ### # # # #########".toCharArray(),
                "#   #     # #     # #   #   # # #       #".toCharArray(),
                "# # ####### ### ### ##### ### # ####### #".toCharArray(),
                "# #             #   #     #   #   #   # #".toCharArray(),
                "# ############### ### ##### ##### # # # #".toCharArray(),
                "#               #     #   #   #   # #   #".toCharArray(),
                "##### ####### # ######### # # # ### #####".toCharArray(),
                "#   # #   #   # #         # # # #       #".toCharArray(),
                "# # # # # # ### # # ####### # # ### ### #".toCharArray(),
                "# # #   # # #     #   #     # #     #   #".toCharArray(),
                "# # ##### # # ####### # ##### ####### # #".toCharArray(),
                "# #     # # # #   # # #     # #       # #".toCharArray(),
                "# ##### ### # ### # # ##### # # ### ### #".toCharArray(),
                "#     #     #     #   #     #   #   #    ".toCharArray(),
                "#########################################".toCharArray()
        });
        return basicMazes;
    }
}


path.Maze

In [4]:
package path;

import java.util.*;

List<char[][]> mazes = Maze.buildTests();
for ( char [][] c : mazes ) {
    Maze m = new Maze(c);
    List<Character> res = m.find();
    System.out.println(res);
}


[F]
[F, F, F, F, F, F, F, F, R, F]
[B, F, F, F, F, F, F, F, F, R, F]
[R, F, L, F, F, L, F, R, F, F, R, F, L, F, F, L, F, F]
[F, R, F, F, R, F, F, F, R, F, F, F, F, R, F, F, F, F, F, R, F, F, F, F, F, F, R, F, F, F, F, F, F]
[B, F, F, F, F, R, F, F, F, F, L, F, F, R, F, F, L, F, F, L, F, F, F, F, L, F, F, R, F, F, R, F, F, F, F, F, F, R, F, F, R, F, F, L, F, F, F, F, F, F, L, F, F, F, F, L, F, F, L, F, F, R, F, F, R, F, F, F, F, F, F, R, F, F, F, F, R, F, F, L, F, F, L, F, F, F, F, L, F, F, R, F, F, F, F, L, F, F, R, F, F, L, F, F, R, F, F, R, F, F, F, F, R, F, F, L, F, F, L, F, F, R, F, F, F, F, F, F, F, F, L, F, F, L, F, F, R, F, F, F, F, F, F, L, F, F, R, F, F, R, F, F, F, F, L, F]


null